# Draw with replacement

Imagin we have an urn, filled with numbered balls. We draw a number of these balls, but after each draw, we place the drawn ball back in the urn. How many different numbers will we have seen after a given number of draws?

To make this precise, imagine the urn contains $N$ balls, labeled with the numbers $0, 1, ..., N$. Let us refer to $N$ as the _urn size_. We draw $D$ balls, i.e. the _draw size_, but after every draw, we place the ball back in the urn. I.e. for each of the $D$ draws, we are effectively sampling from all of the $N$ balls. How many different numbers do we expect to have seen in a draw of size $D$? Let us call this quantity $U$, for the _unique numbers_ we see in the draw.

In [1]:
import numpy as np

## Setting urn and sample size

In [2]:
N = 5  # Urn size
D = 5  # Draw size

## A first implementation
Before trying to calculate how many different numbers we expect to see, let us write a function which performs the experiment a given number of times, and computes the average.

In [3]:
def compute_expected_sample_size(urn_size: int, draw_size: int, no_samples: int = 1_000) -> float:
    """Computes the expected sample size, given the urn size and sample size, by simulating the 
    experiment ``no_samples`` times."""
    # The urn is represented as a list of numbers 0, 1, ..., urn_size - 1
    urn = list(range(urn_size))
    
    # Run the experiment
    diff_numbers_in_sample = []
    for _ in range(int(no_samples)):
        sample = np.random.choice(urn, draw_size, replace=True)
        diff_numbers = len(set(sample))
        diff_numbers_in_sample.append(diff_numbers)
        
    # Compute and return mean
    return np.mean(diff_numbers_in_sample)    

In [4]:
# Let us run a single experiment
%time print(compute_expected_sample_size(N, D, 1000))

3.359
CPU times: user 25.5 ms, sys: 310 µs, total: 25.8 ms
Wall time: 22.2 ms


## Making it a bit faster

The code above is relatively slow. ~25 ms for 1000 samples (on the machine this is run on).
The main suspect for this is the sample generation in every loop. Luckily, ``np.random.choice`` 
allows to specify a multidimentisonal ``size``, which cab efficiently generate all the samples
at once.

In [5]:
def compute_expected_sample_size_fast(urn_size: int, draw_size: int, no_samples: int = 1_000) -> float:
    """Computes the expected sample size, given the urn size and sample size, by simulating the 
    experiment ``no_samples`` times."""
    # The urn is represented as a list of numbers 0, 1, ..., urn_size - 1
    urn = list(range(urn_size))
    
    # Run the experiment
    all_samples = np.random.choice(urn, (no_samples, draw_size), replace=True)
    
    # Compute length, return average
    def no_unique(arr: list) -> int:
        return len(set(arr))
    
    return np.mean(list(map(no_unique, all_samples)))

In [6]:
no_samples = 100000
N = 5
D = 5
%time print(compute_expected_sample_size(N, D, no_samples))
%time print(compute_expected_sample_size_fast(N, D, no_samples))

3.35977
CPU times: user 1.85 s, sys: 88.4 ms, total: 1.94 s
Wall time: 1.81 s
3.36104
CPU times: user 111 ms, sys: 0 ns, total: 111 ms
Wall time: 110 ms


## Analytical solution

The analytical solution can be found through a recursive formula. 

The expected value of the unique numbers for the $n$-th ball drawn $U_n$, given that we know how many 
unique numbers we know so far (i.e. we know $U_{n-1}$) is expressed as:

$$
\begin{align}
    \mathbb{E}\left[ U_{n} | U_{n - 1} \right]  
    & = \frac{U_{n-1}}{N}   U_{n-1} + \frac{N - U_{n-1}}{N}  (U_{n-1} + 1) \\[10 pt]
    & = U_{n-1} + \frac{N - U_{n-1}}{N}.
\end{align}
$$

After some handwaving, we can drop the expectation values (or rather add them at both sides of the equation), leading us to the following equation:

$$
U_{n} = U_{n-1} + \frac{N - U_{n-1}}{N}.
$$

Two observations allow to easily solve this equation: $U_0 = 0$, and $N$ is a fixed point of this recursion relation (i.e. if $U_{n-1} = N$, then also $U_n = N$). 
To solve the recursion relation, define:
$$
\tilde U_n \equiv U_n - N.
$$
Not that this definition recasts the variables as measuring "how far from the fixed point" they are.

The recursion relation then becomes easy to solve:

$$
\begin{align}
\tilde U_{n} + N  & = \left( \tilde U_{n-1} + N\right)  + \frac{N - \left(\tilde U_{n-1} + N \right)}{N} \\
& \Updownarrow \\
\tilde U_{n}  & = \frac{N - 1}{N} \tilde U_{n-1}   \\
& \Updownarrow \\
\tilde U_{n}  & = \left( \frac{N - 1}{N} \right)^n \;  \tilde U_{0}.
\end{align}
$$

Recasting this back in the original variables leads to:
$$
\begin{align}
 U_{n} - N & = \left( \frac{N - 1}{N} \right)^n \;  \left( U_{0} - N \right) \\[10 pt]
& \Updownarrow \quad (U_0 = 0) \\[10 pt]
 U_{n}  & = \left[1 - \left( \frac{N - 1}{N} \right)^n \right]  N . \\
\end{align}
$$


In [7]:
def compute_expected_sample_size_analytical(urn_size: int, draw_size: int) -> float:
    return (1 - ( (urn_size - 1) / urn_size  )**(draw_size)) * urn_size

In [45]:
no_samples = 1000
N = 10000
D = 100
print(compute_expected_sample_size_analytical(N, D))
%time print(compute_expected_sample_size_fast(N, D, no_samples))

99.50661308628095
99.522
CPU times: user 16.6 ms, sys: 109 µs, total: 16.7 ms
Wall time: 14.5 ms
